In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn import linear_model

In [2]:
data = pd.read_excel("C:\\Users\\fl9768\\Desktop\\Data for ML\\Fatigue\\Loc-1\\Extending - Loc - 1_train.xlsx")

In [3]:
data.head()

,Mast name,Load case,Extendingforce,Sectionmodulus,Mastdepth,Sectionmodulus2,Stressvalue
0,DR410i SP,Extending,116140.0,8.472106,28,4.396883,18452
1,DR410i SP,1.1 Extending,127754.0,8.472106,28,4.396883,20212
2,DR410i SP,0.75 Extending,87105.0,8.472106,28,4.396883,13777
3,DR410i SP,0.8 Extending,92912.0,8.472106,28,4.396883,14679
4,DR410i SP,0.9 Extending,104526.0,8.472106,28,4.396883,16515


In [4]:
data = data.drop('Mast name', axis = 'columns')
data = data.drop('Load case', axis = 'columns')

data.head()

,Extendingforce,Sectionmodulus,Mastdepth,Sectionmodulus2,Stressvalue
0,116140.0,8.472106,28,4.396883,18452
1,127754.0,8.472106,28,4.396883,20212
2,87105.0,8.472106,28,4.396883,13777
3,92912.0,8.472106,28,4.396883,14679
4,104526.0,8.472106,28,4.396883,16515


In [5]:
#import smogn

In [6]:
# rg_mtrx = [
#  #   [24000,1,0],
#     [30000,1,0],
#  #   [20000,1,0],
# #   [15000,1,0],
# #     [20000,1,0],
#  #    [21000,1,0],
# #     [21500,1,0],
# #     [22000,1,0],
# #     [22500,1,0],
# #     [23000,1,0],
# #     [23500,1,0],
# #     [24000,1,0],
# #     [24500,1,0],
# #     [25000,1,0],
#  #    [25500,1,0],
# #     [26000,1,0],
# #     [26500,1,0],
# #     [27000,1,0],
# #     [27500,1,0],
#     [10000,0,0],
#     [36000,1,0],
# #     [37000,0,0]
# ]

# # rg_mtrx = [

# #     [8000, 1, 0],  ## over-sample ("minority")
# #     [8500, 1, 0],
# #     [9000, 1, 0],
# #     [9500, 1, 0],
# # #     [10000, 1, 0],
# # #     [10500, 1, 0],
# # #     [11000, 1, 0],
# # #     [11500, 1, 0],
# # #     [12000, 1, 0],
# # #     [12500, 1, 0],
# # #     [13000, 1, 0],
# # #     [13500, 1, 0],
# # #     [14000, 1, 0],
# # #     [14500, 1, 0],
# # #     [15000, 1, 0],
# # #     [15500, 1, 0],
# # #     [16000, 1, 0],
# # #     [16500, 1, 0],
# # #     [17000, 1, 0],
# # #     [17500, 1, 0],
# # #     [18000, 1, 0],
# # #     [18500, 1, 0],
# # #     [19000, 1, 0],
# # #     [19500, 1, 0],
# # #     [20000, 1, 0],
# # #     [20500, 1, 0],
# # #     [21000, 1, 0],
# # #     [11000, 1, 0],
# # #     [12500, 1, 0],
# # #     [18000, 1, 0],
# # #     [19000, 1, 0],
# # #     [20000, 1, 0],
# #     [28000, 0, 0]
# # ]

In [7]:
# data_smogn = smogn.smoter(data = data, y = "Stressvalue", rel_method = 'manual', rel_ctrl_pts_rg = rg_mtrx, samp_method = 'extreme')

In [8]:
#data_smogn

In [9]:
#data.shape

In [10]:
#data_smogn.shape

In [11]:
# frames = [data, data_smogn]
# df = pd.concat(frames)
# df

In [12]:
X_train = data[["Extendingforce", "Sectionmodulus", "Mastdepth"]]
y_train = data[["Stressvalue"]]
test_data = pd.read_excel("C:\\Users\\fl9768\\Desktop\\Data for ML\\Fatigue\\Loc-1\\Extending - Loc - 1_test.xlsx")
X_test = test_data[["Extendingforce", "Sectionmodulus", "Mastdepth"]]
y_test = test_data[["Stressvalue"]]

In [13]:
base_learners = [('xgb1', xgb.XGBRegressor(max_depth = 3, n_estimators = 3000, learning_rate = 0.05, 
                                           gamma = 0.9, random_state = 100)),
                ('xgb2', xgb.XGBRegressor(max_depth = 9, n_estimators = 2000, learning_rate = 0.25, 
                                           gamma = 0.9, random_state = 100)),
                ('rf1', RandomForestRegressor(max_depth = 3, n_estimators = 175, random_state = 100)),
                ('rf2', RandomForestRegressor(max_depth = 10, n_estimators = 175, random_state = 100)),
                ('lr', linear_model.LinearRegression())]

In [14]:
final_reg = StackingRegressor(estimators = base_learners, final_estimator = linear_model.LinearRegression())
final_reg.fit(X_train, y_train.values.ravel())
y_pred_final_reg = final_reg.predict(X_test)
print('RMSE: {}'.format(np.sqrt(mean_squared_error(y_test, y_pred_final_reg))))

RMSE: 1105.6208624532726


In [15]:
y_test['Predicted value stack'] = y_pred_final_reg
y_test['Difference'] = abs(y_test['Stressvalue'] - y_test['Predicted value stack'])
y_test

C:\Users\fl9768\AppData\Local\Temp\ipykernel_7148\339197596.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test['Predicted value stack'] = y_pred_final_reg
C:\Users\fl9768\AppData\Local\Temp\ipykernel_7148\339197596.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test['Difference'] = abs(y_test['Stressvalue'] - y_test['Predicted value stack'])


,Stressvalue,Predicted value stack,Difference
0,22025,20241.679950,1783.320050
1,14229,14128.404366,100.595634
2,11364,12871.044891,1507.044891
3,23055,23683.100696,628.100696
4,12691,12910.979108,219.979108
5,17389,16791.989107,597.010893
6,13076,12526.310944,549.689056
7,8556,10349.366775,1793.366775


In [18]:
import pickle
pickle.dump(final_reg, open('extending_loc_1_stack.pkl', 'wb'))

In [19]:
extending_loc_1_prediction = pickle.load(open('extending_loc_1_stack.pkl', 'rb'))
extending_loc_1_prediction.predict(X_test)

array([20241.67995006, 14128.40436625, 12871.04489122, 23683.10069575,
       12910.97910837, 16791.98910691, 12526.31094444, 10349.36677491])